In [1]:
## Import methods from
%load_ext autoreload

In [2]:
%reload_ext autoreload
from hessenberg import *
from scipy.linalg import hessenberg as hessen
from hessentestcase import hessenTestCase
from collections.abc import Iterable
import matplotlib.pyplot as plt
from itertools import chain

## Hessenberg Form Matrix

In [3]:
## Calculating different matrix for computing hessenberg form

## test matrix
A = np.array([[1,2,3,4], [5,6,7,8], [9, 8, 7, 6], [5,4, 3, 2]], dtype = float)
#print(hessen(A))
A0=hessen(A)

In [4]:
## comparing the hessen output with implemented algorithm
testcase = hessenTestCase(A)
testcase.compare_bound(1*10**-10)

Calculated hessenberg form is of order 1e-10 lower


In [5]:
n = 5
A = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        A[i,j] = 1/(i+1 + j+1 - 1)

testcase = hessenTestCase(A)
testcase.compare_bound(1*10**-10)

Calculated hessenberg form is of order 1e-10 lower


## Converging the hessenberg form matrix to diagonal


* Research algorithms for phase 2

In [6]:
import numpy as np
from numpy.linalg import qr

In [7]:
a = np.array([[0, 2], 
              [2, 3]])
p = [1, 5, 10, 20]
for i in range(20):
    q, r = qr(a)
    a = np.dot(r, q)
    if i+1 in p:
        print(f'Iteration {i+1}:')
        print(a)

Iteration 1:
[[3. 2.]
 [2. 0.]]
Iteration 5:
[[ 3.99998093  0.00976559]
 [ 0.00976559 -0.99998093]]
Iteration 10:
[[ 4.00000000e+00  9.53674316e-06]
 [ 9.53674316e-06 -1.00000000e+00]]
Iteration 20:
[[ 4.00000000e+00  9.09482285e-12]
 [ 9.09494702e-12 -1.00000000e+00]]


In [8]:
np.linalg.eigvals(a)

array([ 4., -1.])

In [9]:
def householder(A):
    
    '''
    Input: 
        A: matrix to be orthogonalized
        Modifies A internally to create a R upper triangular matrix.
    Return: 
        Q: orthonormal matrix
        R: Upper triangular matrix
    '''
    
    r, c = A.shape
    A = np.copy(A)
    Q = np.zeros((r,c))
    
    v = []
    
    for k in range(c):
        
        x = A[k:r,k]
        e1= np.zeros(len(x)); e1[0] = 1
        vk = np.sign(x)*np.linalg.norm(x,2)*e1 + x
        vk = vk/np.linalg.norm(vk,2)
        v.append(vk)
        A[k:r, k:c] = A[k:r, k:c] - (2*np.outer(vk, vk))@A[k:r, k:c]
     
    ## calculating 
    for i in range(c):
        x = np.zeros(r); x[i] = 1
        for k in reversed(range(0, c)):
            
            vk_ = v[k]
            x[k:r] = x[k:r] - 2*vk_*(vk_.T@x[k:r])
            
        Q[:,i] = x
        
    return Q,A[:c,:c]

In [101]:
def QR_shift(matrix, eps, computed_eigvals,k):
    
    A = matrix.copy()
    if A.shape[0] == 1:
        
        ##appending to eigvals list
#         if A.shape[0] not in computed_eigvals
        #computed_eigvals.append(A[0].item())
        return A[0].item()
    
    while np.abs(np.sum(A[-1,0:-1])) > eps: #or check_deflation(A, eps)[0]:
        
        ## pick a miu
        mu =  A[-1, -1] 
        
        ## QR factorization with householder
        Q, R = householder(A - mu*np.eye(A.shape[0]))
        
        ## Recalculate A; 
        A = R@Q + mu*np.eye(A.shape[0])
        #print(A, '\n')
        ## check deflation and break the matrix.
        b, idx = check_deflation(A, eps)
        if idx != None:
            #print(idx, A.shape)
            ## setting (i,i+1), and (i+1, i) to zero
            A[idx[0], idx[1]] = 0
            A[idx[1], idx[0]] = 0
            
            ## splitting H to subdiagonals
            A1 = A[0:idx[0]+1, 0:idx[0]+1]
            A2 = A[idx[0]+1:A.shape[0], 
                    idx[0]+1:A.shape[0]]
            
            ## recursively performing QR-shift iteration on A1 and A2
            #return QR_shift(A1, eps), QR_shift(A2, eps)
            
            return QR_shift(A1, eps, computed_eigvals, k), QR_shift(A2, eps, computed_eigvals, k)
            
    return

def check_deflation(A, eps):
    ##  check the sub-diagonals < eps
    ## return sub-diagonals index
    
    for i in range(A.shape[0]-1):
        if abs(A[i+1, i]) < eps: 
            return (True, (i, i+1))
    return (False, None)

In [102]:
k =0
computed_eigvals = []
A = np.array([[1,2,4], [2,6,8], [4, 8, 7]], dtype = float)
A0 = hessen(A)
print(QR_shift(A0, 10**-12, computed_eigvals, k))
#computed_eigvals = np.array(sorted(np.array(computed_eigvals)))
inbuilt_eigvals = np.linalg.eigvals(A)                          
print(inbuilt_eigvals)
# np.linalg.norm(computed_eigvals - inbuilt_eigvals,2)

((15.766890392317112, -2.269846387630128), 0.5029559953130315)
[15.76689039  0.502956   -2.26984639]


In [103]:
check_deflation(A, 10**-4)[0]

False

In [104]:
computed_eigvals = []
A = np.array([[1,2,4,5], 
              [2,6,8, 4], 
              [4, 8, 7, 3], 
              [5, 4, 3, 4]
             ], dtype = float)
A0 = hessen(A)
print(QR_shift(A0, 10**-6, computed_eigvals, k))
# computed_eigvals = np.array(sorted(np.array(computed_eigvals)))
inbuilt_eigvals =np.linalg.eigvals(A)
print(inbuilt_eigvals)
# np.linalg.norm(computed_eigvals - inbuilt_eigvals,2)

# plt.plot(range(len(inbuilt_eigvals)), inbuilt_eigvals, label = "input")
# plt.plot(range(len(computed_eigvals)), computed_eigvals, label = "computed")
# plt.legend()

(((18.46928438164622, -3.7863364227905882), 3.7854691088446044), -0.46841706770020464)
[18.46928438  3.78546911 -3.78633642 -0.46841707]


In [105]:
A0

array([[ 1.00000000e+00, -6.70820393e+00, -4.30694199e-16,
         4.32032588e-16],
       [-6.70820393e+00,  1.25333333e+01, -6.84462482e+00,
        -2.62316179e-15],
       [ 0.00000000e+00, -6.84462482e+00,  4.24904026e+00,
        -2.24900794e+00],
       [ 0.00000000e+00,  0.00000000e+00, -2.24900794e+00,
         2.17626411e-01]])

In [106]:
H1 = np.array([[8, 11, 4, 3], 
              [11, 12, 4, 7], 
              [4, 4, 7, 12], 
              [3, 7, 12, 17]])

U = np.random.uniform(low=0, high=1.0, size=(10, 10))
S = np.tril(U) + np.tril(U, -1).T
A0 = hessen(S)
computed_eigvals = []
print(QR_shift(A0, 10**-6, computed_eigvals, k))
# computed_eigvals = np.array(sorted(np.array(computed_eigvals)))
# print(len(np.linalg.eigvals(S)))
# print(len(computed_eigvals))
# inbuilt_eigvals = np.array(sorted(np.linalg.eigvals(S)))
# print(computed_eigvals)
# np.linalg.norm(computed_eigvals - inbuilt_eigvals,2)

((((((5.459739355408075, (((-1.3430731577465287, 1.3462280682657433), -0.7170293233731412), -0.41095912203925494)), 0.8997956846148463), 0.7771138494307779), 0.5183054478418537), 0.1304691685009324), 0.13960901483784463)


In [107]:
A

array([[1., 2., 4., 5.],
       [2., 6., 8., 4.],
       [4., 8., 7., 3.],
       [5., 4., 3., 4.]])

In [15]:
print(unravel(result))

NameError: name 'unravel' is not defined

In [21]:
A

array([[2, 4]])

In [22]:
A.item()

ValueError: can only convert an array of size 1 to a Python scalar